In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import load_model

In [2]:
BASE_PATH = '/input/understanding_cloud_organization/'
TRAIN_IMAGE = '/input/understanding_cloud_organization/train_images'
TEST_IMAGE = '/input/understanding_cloud_organization/test_images'

In [ ]:
bad_image = ['046586a.jpg', '1588d4c.jpg', '1e40a05.jpg', '41f92e5.jpg', '449b792.jpg',
             '563fc48.jpg', '8bd81ce.jpg', 'c0306e5.jpg', 'c26c635.jpg', 'e04fea3.jpg',
             'e5f2f24.jpg', 'eda52f2.jpg', 'fa645da.jpg']
train_df = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
train_df['Image'] = train_df['Image_Label'].apply(lambda x: x.split('_')[0])
train_df['Label'] = train_df['Image_Label'].apply(lambda x: x.split('_')[1])
train_df['has_mask'] = ~train_df['EncodedPixels'].isna()
train_df=train_df[~train_df['Image'].isin(bad_image)]

In [ ]:
mask_count_df = train_df.groupby('Image').agg(np.sum).reset_index()

# Utility fuctions

In [3]:
#post process by threshold and mini_mask, with rectangle funtion(optional)
def post_process(mask, threshold, min_size, input_size=(350, 525), image=None, rectangle=False):
    new_mask = np.zeros(input_size, dtype='uint8')
    blk_mask = np.empty(input_size, dtype='uint8')
    mask = cv2.threshold(mask, threshold, 1, cv2.THRESH_BINARY)[1] #threshold
    
    num_component, component = cv2.connectedComponents(mask.astype(np.uint8)) #component split

    predictions = np.zeros(input_size, np.float32)
    num = 0
    
    for c in range(1, num_component):
        p = (component == c)
        #remove mask component if size < mini_size
        if p.sum() > min_size:
            #if apply, transform mask shape to rectanle
            if rectangle:
                new_mask = rectangle_process(image, new_mask, p)
            predictions[p] = 1
            num += 1
    return predictions, new_mask, num

def rectangle_process(img, new_mask, block):
    cnts, _ = cv2.findContours(block.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        cv2.rectangle(new_mask, (x, y), (x+w, y+h), 1, -1)
        l = np.array([0, 0, 0])
        u = np.array([0, 0, 0])
        blk_mask = cv2.inRange(img, l, u)
        new_mask = cv2.subtract(new_mask, blk_mask)
        return new_mask

In [4]:
def np_resize(img, input_shape):
    height, width = input_shape
    return cv2.resize(img, (width, height))

def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def mask_maker(rles, input_shape, reshape=None):
    depth = len(rles)
    if reshape is None:
        masks = np.zeros((*input_shape, depth))
    else:
        masks = np.zeros((*reshape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            if reshape is None:
                masks[:, :, i] = rle2mask(rle, input_shape)
            else:
                mask = rle2mask(rle, input_shape)
                reshaped_mask = np_resize(mask, reshape)
                masks[:, :, i] = reshaped_mask
    
    return masks

def build_rles(masks, reshape=None):
    width, height, depth = masks.shape
    
    rles = []
    
    for i in range(depth):
        mask = masks[:, :, i]
        
        if reshape:
            mask = mask.astype(np.float32)
            mask = np_resize(mask, reshape).astype(np.int64)
        
        rle = mask2rle(mask)
        rles.append(rle)
        
    return rles

In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

# DataGenerator

In [6]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, list_IDs, df, target_df, batch_size, mode='fit',
                base_path='/input/understanding_cloud_organization/train_images',
                channels=3, classes=4, input_size=(1400, 2100) ,augment=False, gamma=None,
                reshape=None, random_state=2019, shuffle=True):
    
        self.list_IDs = list_IDs
        self.df = df
        self.target_df = target_df
        self.batch_size = batch_size
        self.mode = mode
        self.base_path = base_path
        self.channels = channels
        self.classes = classes
        self.input_size = input_size
        self.augment = augment
        self.reshape = reshape
        self.random_state = random_state
        self.shuffle = shuffle
        self.gamma = gamma

        self.on_epoch_end()
        np.random.seed(self.random_state)
    
    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size: (index + 1) * self.batch_size]
        
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generator_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generator_y(list_IDs_batch)
            
            if self.augment:
                X, y = self.__batch_augment(X, y)
            
            return X, y
        
        elif self.mode == 'predict':
            return X
        
        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
            
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generator_X(self, list_IDs_batch):
        
        if self.reshape is not None:
            X = np.empty((self.batch_size, *self.reshape, self.channels))
        else:
            X = np.empty((self.batch_size, *self.input_size, self.channels))
        
        for i, ID in enumerate(list_IDs_batch):
            fn = self.df['Image'].iloc[ID]
            img_path = f'{self.base_path}/{fn}'
            img = self.__load_rgb(img_path)
            
            if self.reshape is not None:
                img = np_resize(img, self.reshape)
            if self.gamma is not None:
                img = adjust_gamma(img, gamma=self.gamma)
            
            X[i, ] = img
            
        return X
    
    def __generator_y(self, list_IDs_batch):
        
        if self.reshape is not None:
            y = np.empty((self.batch_size, *self.reshape, self.classes), dtype=int)
        else:
            y = np.empty((self.batch_size, *self.input_size, self.classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            img = self.df['Image'].iloc[ID]
            img_df = self.target_df[self.target_df['Image'] == img]
            rles = img_df['EncodedPixels']
            
            if self.reshape is None:
                mask = mask_maker(rles, input_shape=self.input_size)
            else:
                mask = mask_maker(rles, input_shape=self.input_size, reshape=self.reshape)
                        
            y[i, ] = mask
            
        return y
            
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)
        return img
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.
        return img
    
    def __random_transform(self, img, masks):
        composition = albu.Compose([
            albu.HorizontalFlip(),
            albu.VerticalFlip(),
            albu.ShiftScaleRotate(shift_limit=0.1, rotate_limit=30),
            albu.OneOf([
                albu.GridDistortion(p=0.5),
                albu.OpticalDistortion(p=0.5, distort_limit=2, shift_limit=0.5),
            ], p=0.5)
        ])
        
        composed = composition(image=img, mask=masks)
        aug_img = composed['image']
        aug_mask = composed['mask']
        return aug_img, aug_mask
    
    def __batch_augment(self, img_batch, mask_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i,], mask_batch[i, ] = self.__random_transform(img_batch[i, ], mask_batch[i, ]) 
        return img_batch, mask_batch

In [7]:
#read submission csv and test images
sub_df = pd.read_csv('/input/understanding_cloud_organization/sample_submission.csv')
sub_df['Image'] = sub_df['Image_Label'].apply(lambda x: x.split('_')[0])
test_imgs = pd.DataFrame(sub_df['Image'].unique(), columns=['Image'])

# Ensemble trained models
Consider we dont have enought computing resourse, therefore I training each model sepratly and ensemble all model's predictions and calculate it for my final predictions.

In [8]:
all_models = ['model_DenseNet169_fpn_n2.h5','model_resnet101_unet_sm.h5', 
              'model_incepv3_fpn_n2.h5', 'model_res34_sm.h5']

In [9]:
#ensemble multiple model
ensemble = np.empty((3698, 320, 480, 4), dtype=np.float16)
for i in range(len(all_models)):
    print('running: {}'.format(all_models[i]))
    model = None
    model = load_model(all_models[i],
                       custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef': dice_coef})
    test_df = []

    for i in range(0, test_imgs.shape[0], 100):
        batch_idx = list(
            range(i, min(test_imgs.shape[0], i + 100))
        )

        test_generator = DataGenerator(
            batch_idx,
            df=test_imgs,
            shuffle=False,
            mode='predict',
            input_size=(350, 525),
            reshape=(320, 480),
            #gamma=0.8,
            channels=3,
            base_path=TEST_IMAGE,
            target_df=sub_df,
            batch_size=1,
            classes=4
        )

        batch_pred_masks = model.predict_generator(
            test_generator, 
            workers=1,
            verbose=1
        )

        ensemble[i:i+len(batch_idx), ] = np.add(ensemble[i:i+len(batch_idx), ], batch_pred_masks*0.25)
    
    keras.backend.clear_session()

running: model_DenseNet169_fpn_n2.h5


W1107 11:55:14.053850  8348 deprecation.py:323] From C:\Users\zessi\AppData\Local\anaconda3\envs\tf-20-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


98/98 [==============================] - 10s 104ms/step
running: model_resnet101_unet_sm.h5
98/98 [==============================] - 10s 104ms/step
running: model_incepv3_fpn_n2.h5
98/98 [==============================] - 10s 104ms/step
running: model_res34_sm.h5
98/98 [==============================] - 9s 89ms/step


In [41]:
#save ensemble model's predictions to npy
print(ensemble.shape)
np.save("4fold_ensemble", ensemble)

(3698, 320, 480, 4)


# Predict and Submission (w/o post-process)

In [12]:
#predict on test sets
test_df = []
for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )

    for j, idx in enumerate(batch_idx):
        filename = test_imgs['Image'].iloc[idx]
        image_df = sub_df[sub_df['Image'] == filename].copy()
        pred_masks = ensemble[i+j, ].round().astype(int)

        pred_rles = build_rles(pred_masks, reshape=(350, 525))

        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)

In [13]:
#save predictions for submission
test_df = pd.concat(test_df)
pre_df = test_df.copy()
test_df.drop(columns='Image', inplace=True)
test_df.to_csv('submission_v40_ensemble_v2.csv', index=False)

# Predict and Submission with Post-process

In [38]:
#set threshold and minsize mask to each cloud category
thresholds = [0.6, 0.40, 0.55, 0.40]
minsizes = [10000, 15000, 10000, 15000]

In [39]:
test_df = []
for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )

    for j, idx in enumerate(batch_idx):
        filename = test_imgs['Image'].iloc[idx]
        image_df = sub_df[sub_df['Image'] == filename].copy()
        #img = cv2.imread(f'{TEST_IMAGE}/{filename}')
        #img = np_resize(img, (320, 480))
        
        pred_masks = np.empty((320, 480, 4))
        for n in range(4):  
            pred_mask, _, _ = post_process(ensemble[i+j,:,:,n].astype(float) , threshold=thresholds[n],
                                           min_size=minsizes[n],
                                           input_size=(320, 480),
                                           rectangle=False)
            pred_masks[:,:,n] = pred_mask 
            
        pred_rles = build_rles(pred_masks, reshape=(350, 525))

        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)        

In [40]:
test_df = pd.concat(test_df)
pre_df = test_df.copy()
test_df.drop(columns='Image', inplace=True)
test_df.to_csv('submission_v42_ensemble_v2_posted_v11.csv', index=False)

# Post-Process parameter optimization

In [12]:
from sklearn.model_selection import train_test_split
train_idx, val_idx = train_test_split(mask_count_df.index, random_state=2019, test_size=0.2)

In [13]:
#Use multiple model to predict validation sets 
val_ensemble_pred_masks = np.empty((1107, 320, 480, 4))
for i in range(len(all_models)):
    print('running: {}'.format(all_models[i]))
    model = None
    model = load_model(all_models[i],
                       custom_objects={'bce_dice_loss': bce_dice_loss, 'dice_coef': dice_coef})
    test_df = []
    for i in range(0, val_idx.shape[0], 100):
        batch = list(
            range(i, min(val_idx.shape[0], i + 100))
        )
        batch_val_idx = val_idx[i:i+len(batch)]

        optimal_generator = DataGenerator(batch_val_idx,
                                  df=mask_count_df,
                                  target_df=train_df,
                                  batch_size=1,
                                  mode='predict',
                                  shuffle=False,
                                  augment=False,
                                  #gamma=0.8,
                                  reshape=(320, 480)
                                 )

        val_pred_masks = model.predict_generator(
            optimal_generator,
            workers=1,
            verbose=1
        )

        val_ensemble_pred_masks[i:i+len(batch), ] = np.add(val_ensemble_pred_masks[i:i+len(batch), ], val_pred_masks/3)
    
    keras.backend.clear_session()

running: model_DenseNet169_fpn_n2.h5


W1107 00:37:05.168400  5064 deprecation.py:323] From C:\Users\zessi\AppData\Local\anaconda3\envs\tf-20-gpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


7/7 [==============================] - 1s 125ms/step
running: model_resnet101_unet_sm.h5
7/7 [==============================] - 1s 119ms/step
running: model_incepv3_fpn_n2.h5
7/7 [==============================] - 1s 128ms/step
running: model_res34_sm.h5
7/7 [==============================] - 1s 127ms/step


In [14]:
def val_dice_coef(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

In [16]:
from tqdm import tqdm

#img = np.empty((len(val_idx), 320, 480, 3))
true_y = np.empty((len(val_idx),320, 480, 4))
#Generator img and true masks
for i in tqdm(range(len(val_idx))):           
    fn = mask_count_df['Image'].iloc[val_idx[i]]
   # image = cv2.imread(f'{TRAIN_IMAGE}/{fn}')
    #img[i, ] = np_resize(image, (320, 480))
    
    rles_df = train_df[train_df['Image'] == fn]
    rles = rles_df['EncodedPixels']
    true_y[i, ] = mask_maker(rles, input_shape=(1400, 2100), reshape=(320, 480))

100%|██████████████████████████████████████████████████████████████████████████████| 1107/1107 [01:34<00:00, 11.70it/s]


# Grid search 

In [17]:
from tqdm import tqdm
cate_label = {0: 'Fish', 1: 'Flower', 2: 'Gravel', 3: 'Sugar'}
val_dice = []

for c in tqdm(range(4)):
    for t in range(30, 80, 5):
        t = t/100
        for mi in [5000, 10000, 15000, 20000]:
            pred_y = np.empty((320, 480))
            coef = []
            for i in range(len(val_idx)):           
                pred_y, _, _ = post_process(val_ensemble_pred_masks[i,:,:,c], threshold=t,
                                            min_size=mi, input_size=(320, 480),
                                            #image=img[i, ],
                                            rectangle=False)              
                coef.append(val_dice_coef(true_y[i, :, :, c], pred_y))
            coef = np.mean(coef)
            val_dice.append([t, mi, coef, cate_label[c]])
            #print('{}, {}, {}, ceof = {}'.format(c, t, mi, coef))

100%|█████████████████████████████████████████████████████████████████████████████████| 4/4 [1:04:01<00:00, 974.03s/it]


In [18]:
opt_df = pd.DataFrame(val_dice, columns=['threshold', 'min_size', 'dice_coef', 'cate'])
opt_df.to_csv('parameter_optimal_ensembl_v2.csv', index=False)

for i in range(4):
    print(opt_df[opt_df['cate'] == cate_label[i]].sort_values(by='dice_coef', ascending=False).head())

    threshold  min_size  dice_coef  cate
23       0.55     20000   0.636998  Fish
39       0.75     20000   0.633754  Fish
27       0.60     20000   0.633318  Fish
35       0.70     20000   0.631514  Fish
34       0.70     15000   0.631149  Fish
    threshold  min_size  dice_coef    cate
74       0.70     15000   0.774896  Flower
78       0.75     15000   0.774379  Flower
62       0.55     15000   0.773696  Flower
66       0.60     15000   0.772655  Flower
58       0.50     15000   0.772501  Flower
     threshold  min_size  dice_coef    cate
114       0.70     15000   0.636006  Gravel
107       0.60     20000   0.634482  Gravel
111       0.65     20000   0.633479  Gravel
118       0.75     15000   0.632979  Gravel
99        0.50     20000   0.630960  Gravel
     threshold  min_size  dice_coef   cate
141       0.55     10000   0.610706  Sugar
145       0.60     10000   0.608452  Sugar
153       0.70     10000   0.606286  Sugar
149       0.65     10000   0.606216  Sugar
134       0.45   

# Predict and Submission with Post-process

In [38]:
#set optimal threshold and minsize mask to each cloud category
thresholds = [0.55, 0.70, 0.70, 0.55]
minsizes = [20000, 15000, 150000, 10000]

In [39]:
test_df = []
for i in range(0, test_imgs.shape[0], 500):
    batch_idx = list(
        range(i, min(test_imgs.shape[0], i + 500))
    )

    for j, idx in enumerate(batch_idx):
        filename = test_imgs['Image'].iloc[idx]
        image_df = sub_df[sub_df['Image'] == filename].copy()
        #img = cv2.imread(f'{TEST_IMAGE}/{filename}')
        #img = np_resize(img, (320, 480))
        
        pred_masks = np.empty((320, 480, 4))
        for n in range(4):  
            pred_mask, _, _ = post_process(ensemble[i+j,:,:,n].astype(float) , threshold=thresholds[n],
                                           min_size=minsizes[n],
                                           input_size=(320, 480),
                                           rectangle=False)
            pred_masks[:,:,n] = pred_mask 
            
        pred_rles = build_rles(pred_masks, reshape=(350, 525))

        image_df['EncodedPixels'] = pred_rles
        test_df.append(image_df)        

In [40]:
test_df = pd.concat(test_df)
pre_df = test_df.copy()
test_df.drop(columns='Image', inplace=True)
test_df.to_csv('submission_v42_ensemble_v2_posted_v11.csv', index=False)